In [176]:
import sys
import pickle
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectPercentile,f_classif
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn.linear_model import SGDClassifier

In [128]:
test_file = pd.read_csv("/guibrunhole/unicamp/mc866/p1/year-prediction-msd-test.txt")
train_file = pd.read_csv("/guibrunhole/unicamp/mc866/p1/year-prediction-msd-train.txt")

In [7]:
test_file.head()

,1989,41.50753,8.53702,15.18306,3.57066,-2.72349,0.14377,-10.43850,-5.76573,12.94180,...,13.36551,-215.67790,111.89802,99.40722,-8.92967,238.77197,-45.18287,16.28315,59.43443,-1.93970
0,1989,42.04629,104.35074,-35.26407,-18.36078,22.00406,-11.81382,-13.55492,-5.20432,3.54712,...,-2.04685,-109.89799,-54.27301,18.63117,-20.82212,64.52874,-113.12215,-6.01389,50.07127,-12.21698
1,1987,47.55231,30.38790,10.40024,3.56833,11.29946,0.16833,-13.83527,-6.63428,5.44989,...,-5.36013,-33.66620,-72.68807,42.37045,7.91002,18.09998,-77.91514,0.53547,-43.80638,-11.46310
2,2004,39.18995,32.40987,3.41234,-9.13243,-13.13339,-2.24082,-13.64218,2.88199,3.17845,...,20.86701,-224.29996,133.90916,-226.36115,3.55048,156.74744,-865.32391,4.66481,-323.98246,9.53698
3,2001,44.88043,-127.37818,52.30306,3.33231,-15.62723,-19.69058,11.84617,-2.67727,18.25535,...,28.63967,-72.81989,108.48976,-28.81834,-13.61699,-244.72325,-103.33381,-1.91128,106.51424,5.80923
4,2001,49.01346,-11.64058,87.63193,23.66652,-30.26612,-20.76604,3.00914,-7.24580,13.47411,...,33.71977,-55.96769,-106.41607,-4.18955,-4.95720,-141.97314,-85.01688,14.83452,-22.46561,8.59966


In [8]:
train_file.head()

,2004,46.50128,-18.37096,58.89824,27.28181,1.93097,-2.43167,18.64843,-5.62769,-1.47711,...,43.94716,-64.21148,13.04284,156.03785,-2.42670,51.71977,-43.56703,10.63735,24.08962,-21.41886
0,2005,41.46015,-52.30140,-4.59825,-19.28084,-11.85844,-19.54192,1.30306,-1.83185,5.98469,...,-13.48435,-154.65715,35.22429,89.53649,-3.13145,-128.28120,89.97277,-15.88139,-75.21074,-0.51139
1,2009,42.18667,-85.65863,-7.91506,-11.84193,-12.61959,-21.66749,4.97883,-11.86750,9.87342,...,42.08584,-181.77349,43.00181,87.94977,-13.70362,136.49979,140.32633,11.51422,382.79589,29.98269
2,1987,46.90244,19.86314,10.92119,4.87136,-41.17499,-19.84156,2.93308,-5.98711,3.05997,...,-2.76407,-20.31782,-75.24506,125.81801,-15.50828,-63.31002,-142.21937,-12.36699,32.45911,-17.14909
3,2009,46.64388,-81.99503,41.62851,3.65855,-9.38201,-13.51749,7.48781,-7.03302,7.06982,...,12.29244,-143.25348,91.15842,-227.85481,7.75916,-41.32376,-225.66526,-4.05081,455.39458,41.65310
4,2010,40.08240,-106.24846,41.81955,2.03363,-16.44993,-2.90037,-22.26724,-2.94540,11.63992,...,65.00995,-190.08468,-94.97429,-9.73645,-3.29013,51.91596,-320.37894,-5.58047,83.48503,-4.19480


In [9]:
print 'Total de registros no arquivo de teste: ', len(test_file)
print 'Total de registros no arquivo de treino: ', len(train_file)

Total de registros no arquivo de teste:  36284
Total de registros no arquivo de treino:  463714


In [10]:
train_file.columns

Index([u'2004', u'46.50128', u'-18.37096', u'58.89824', u'27.28181',
       u'1.93097', u'-2.43167', u'18.64843', u'-5.62769', u'-1.47711',
       u'-4.15271', u'2.53346', u'4.81844', u'57.29647', u'1494.49559',
       u'1744.23604', u'2068.65394', u'1186.51553', u'1284.21970',
       u'538.97998', u'412.62502', u'339.02682', u'336.69194', u'269.99563',
       u'273.31841', u'-37.62561', u'-315.49924', u'-471.11338', u'-81.25060',
       u'30.07995', u'116.20072', u'-35.71057', u'-10.85118', u'-7.22255',
       u'10.18667', u'-7.43910', u'110.37861', u'-235.73664', u'267.59686',
       u'-335.67166', u'-164.26158', u'1.66648', u'125.68117', u'-43.84793',
       u'5.00488', u'-42.17875', u'-39.35691', u'333.43226', u'-109.15363',
       u'-196.07567', u'58.86028', u'-53.14465', u'29.50103', u'-4.37532',
       u'-55.98788', u'-20.67896', u'6.83154', u'-94.78818', u'-69.53280',
       u'-31.05901', u'-115.13438', u'-10.49483', u'2.89506', u'-36.88605',
       u'-184.10634', u'-32.69213',

In [129]:
new_columns_names = []

new_columns_names.append('target')

for i in range(len(train_file.columns) -1):
    name = 'column_'+str(i)
    new_columns_names.append(name)

In [130]:
old_columns_names = train_file.columns

train_file.rename(columns=dict(zip(old_columns_names, new_columns_names)), inplace=True)

In [131]:
old_columns_names = test_file.columns

test_file.rename(columns=dict(zip(old_columns_names, new_columns_names)), inplace=True)

In [14]:
train_file.head()

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89
0,2005,41.46015,-52.30140,-4.59825,-19.28084,-11.85844,-19.54192,1.30306,-1.83185,5.98469,...,-13.48435,-154.65715,35.22429,89.53649,-3.13145,-128.28120,89.97277,-15.88139,-75.21074,-0.51139
1,2009,42.18667,-85.65863,-7.91506,-11.84193,-12.61959,-21.66749,4.97883,-11.86750,9.87342,...,42.08584,-181.77349,43.00181,87.94977,-13.70362,136.49979,140.32633,11.51422,382.79589,29.98269
2,1987,46.90244,19.86314,10.92119,4.87136,-41.17499,-19.84156,2.93308,-5.98711,3.05997,...,-2.76407,-20.31782,-75.24506,125.81801,-15.50828,-63.31002,-142.21937,-12.36699,32.45911,-17.14909
3,2009,46.64388,-81.99503,41.62851,3.65855,-9.38201,-13.51749,7.48781,-7.03302,7.06982,...,12.29244,-143.25348,91.15842,-227.85481,7.75916,-41.32376,-225.66526,-4.05081,455.39458,41.65310
4,2010,40.08240,-106.24846,41.81955,2.03363,-16.44993,-2.90037,-22.26724,-2.94540,11.63992,...,65.00995,-190.08468,-94.97429,-9.73645,-3.29013,51.91596,-320.37894,-5.58047,83.48503,-4.19480


In [15]:
test_file.head()

,target,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7,column_8,...,column_80,column_81,column_82,column_83,column_84,column_85,column_86,column_87,column_88,column_89
0,1989,42.04629,104.35074,-35.26407,-18.36078,22.00406,-11.81382,-13.55492,-5.20432,3.54712,...,-2.04685,-109.89799,-54.27301,18.63117,-20.82212,64.52874,-113.12215,-6.01389,50.07127,-12.21698
1,1987,47.55231,30.38790,10.40024,3.56833,11.29946,0.16833,-13.83527,-6.63428,5.44989,...,-5.36013,-33.66620,-72.68807,42.37045,7.91002,18.09998,-77.91514,0.53547,-43.80638,-11.46310
2,2004,39.18995,32.40987,3.41234,-9.13243,-13.13339,-2.24082,-13.64218,2.88199,3.17845,...,20.86701,-224.29996,133.90916,-226.36115,3.55048,156.74744,-865.32391,4.66481,-323.98246,9.53698
3,2001,44.88043,-127.37818,52.30306,3.33231,-15.62723,-19.69058,11.84617,-2.67727,18.25535,...,28.63967,-72.81989,108.48976,-28.81834,-13.61699,-244.72325,-103.33381,-1.91128,106.51424,5.80923
4,2001,49.01346,-11.64058,87.63193,23.66652,-30.26612,-20.76604,3.00914,-7.24580,13.47411,...,33.71977,-55.96769,-106.41607,-4.18955,-4.95720,-141.97314,-85.01688,14.83452,-22.46561,8.59966


In [16]:
test = test_file[new_columns_names].copy().applymap(lambda x: 0  if x == 'NaN' else x) #replace NaN features as 0
train = train_file[new_columns_names].copy().applymap(lambda x: 0  if x == 'NaN' else x) #replace NaN features as 0

# Usando regressão linear

In [55]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [56]:
reg = linear_model.LinearRegression()

### Primeiro vamos fazer utilizando todas as features de nosso conjunto de dados

#### Feature Selection

In [145]:
features_train = train_file.copy()
del features_train['target']
labels_train = train_file['target']

features_test = test_file.copy()
del features_test['target']
labels_test = test_file['target']

In [146]:
reg.fit(features_train, labels_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [140]:
prediction = reg.predict(features_test)

In [147]:
# The coefficients
print('Estimated intercept: ', reg.intercept_)

# The coefficients
print('Coefficients: ', reg.coef_)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

('Estimated intercept: ', 1951.12223733122)
('Coefficients: ', array([  8.73438225e-01,  -5.58164108e-02,  -4.34529202e-02,
         3.51222597e-03,  -1.48555094e-02,  -2.21504166e-01,
        -6.25852689e-03,  -1.01802126e-01,  -6.77198017e-02,
         2.27520902e-02,  -1.64417316e-01,  -1.99906291e-03,
         4.71502740e-02,   3.57496542e-04,  -4.34314442e-04,
         6.12375590e-04,   4.36150084e-04,   1.44957271e-03,
         1.97893348e-03,   2.21197308e-03,   7.29342435e-04,
        -5.51434773e-04,   7.62630491e-03,   2.73358084e-03,
        -3.58577614e-03,   3.76722775e-05,   1.59723630e-03,
         4.75391556e-04,   9.15829856e-04,  -2.38349166e-04,
        -1.24190011e-03,  -1.40863226e-03,  -5.50752995e-03,
         2.57380968e-03,   1.62822657e-03,  -5.20660337e-03,
        -2.47303949e-04,   6.64414964e-04,   1.40559069e-03,
        -1.72540963e-03,  -2.08520137e-03,  -8.10852342e-04,
        -1.51360229e-03,  -2.42147138e-03,  -3.23754125e-03,
         6.67637873e-0

### Agora vamos usar o algoritmo de seleção de features conhecido como SelectPercentile

In [177]:
def select_features(percentile):
    df = train_file.copy()
    del df['target']
    labels_train = train_file['target']

    selPerc = SelectPercentile(f_classif,percentile=percentile) # Built the SelectPercentile
    selPerc.fit(df,labels_train) # Learn the Features, knowing which features to use

    features_percentiled = df.columns[selPerc.get_support()].tolist() #Filter columns based on what Percentile support
    df['target'] = labels_train #rejoin the label

    # features_percentiled # target need to be the first one

    features_train = train_file[features_percentiled].copy()
    labels_train = train_file['target']

    features_test = test_file[features_percentiled].copy()
    labels_test = test_file['target']

    return features_train, labels_train, features_test, labels_test

### Primeiro vamos começar selecionando apenas 25% das features

In [178]:
features_train, labels_train, features_test, labels_test = select_features(25)

## fitting the model
reg.fit(features_train, labels_train)
prediction = reg.predict(features_test)

# The coefficients
print('Estimated intercept: ', reg.intercept_)
# The coefficients
print('Coefficients: ', reg.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

('Estimated intercept: ', 1960.668504569139)
('Coefficients: ', array([  7.21584191e-01,  -3.99502839e-02,  -4.67357795e-02,
        -1.68566651e-01,   3.16454074e-03,  -3.69380335e-02,
         5.37726837e-04,   5.90679484e-03,   3.69503192e-03,
        -2.87675199e-03,  -4.23904820e-03,  -3.27180684e-03,
         7.14595086e-03,   5.32371507e-04,  -3.35115852e-04,
        -1.55096279e-03,  -3.26611622e-03,  -8.28933845e-03,
         2.56842231e-03,  -2.61711003e-03,   7.46794140e-04,
         5.71347883e-03,  -3.46097140e-03]))
Mean squared error: 96.74
Variance score: 0.18


### Agora vamos fazer com 50% das features

In [174]:
features_train, labels_train, features_test, labels_test = select_features(50)

## fitting the model
reg.fit(features_train, labels_train)
prediction = reg.predict(features_test)

# The coefficients
print('Estimated intercept: ', reg.intercept_)
# The coefficients
print('Coefficients: ', reg.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

('Estimated intercept: ', 1954.1852475862217)
('Coefficients: ', array([  8.26554159e-01,  -4.66430777e-02,  -4.18008436e-02,
        -2.24265605e-01,  -1.84595094e-02,  -6.24687833e-02,
        -1.60411550e-01,  -1.05058393e-02,   4.72486033e-04,
         7.13575206e-04,   2.58088811e-03,   2.37983222e-03,
         1.71146759e-03,  -3.54187693e-04,   7.70887594e-03,
         2.15071628e-03,  -3.17667916e-03,   5.01352182e-04,
        -2.23550305e-03,  -4.61463299e-03,  -4.99607675e-03,
         1.05563502e-03,   2.18943079e-03,  -2.23876466e-03,
        -5.43506136e-03,   7.22832994e-03,   4.16539889e-04,
        -1.28831068e-03,   2.48954469e-03,  -1.08723407e-03,
        -1.62177493e-03,  -1.91583016e-03,  -2.13188672e-03,
         2.09885745e-03,  -4.21409459e-03,   7.05803168e-04,
         5.18350040e-04,  -1.02038708e-03,  -4.39235798e-03,
         8.07664087e-04,   1.67130633e-03,   3.80674802e-03,
        -4.62904770e-03,   7.87071964e-04,   1.70988180e-03]))
Mean squared error

### Agora vamos fazer com 75% das features

In [175]:
features_train, labels_train, features_test, labels_test = select_features(75)

## fitting the model
reg.fit(features_train, labels_train)
prediction = reg.predict(features_test)

# The coefficients
print('Estimated intercept: ', reg.intercept_)
# The coefficients
print('Coefficients: ', reg.coef_)
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

('Estimated intercept: ', 1953.1889387276542)
('Coefficients: ', array([  8.32998718e-01,  -5.06612799e-02,  -3.46447678e-02,
        -1.36621110e-02,  -1.42409288e-02,  -2.30328939e-01,
        -8.90331086e-03,  -1.02265481e-01,  -7.08342838e-02,
         1.43695487e-02,  -1.73922138e-01,   3.67708465e-03,
         4.60389512e-04,  -3.94505144e-04,   7.89072787e-04,
         6.06689811e-04,   1.63781620e-03,   2.09428069e-03,
         2.01512617e-03,   1.28011316e-03,  -1.19098305e-03,
         7.94881357e-03,   1.93733885e-03,  -2.77753440e-03,
         1.70916343e-03,   3.64904033e-04,   1.04067966e-03,
        -9.64280943e-04,  -6.57446535e-03,   3.18589253e-03,
        -2.65219986e-03,  -1.86084191e-04,   7.67413286e-04,
         1.69668023e-03,  -2.02773132e-03,  -2.74250234e-03,
        -1.60155681e-03,   4.47928299e-03,   5.47381736e-04,
        -2.28507986e-03,   1.12710743e-04,   2.25089071e-03,
        -1.59345722e-03,   1.58726130e-03,  -1.10715301e-03,
        -2.00843290e

# Usando SGDClassifier

### 25% das features

In [181]:
clf = SGDClassifier()

features_train, labels_train, features_test, labels_test = select_features(25)

clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
acc_lm = clf.score(features_test, labels_test)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

print "Linear Model with 25% of features accuracy is: ", acc_lm

Mean squared error: 163.75
Variance score: -0.38
Linear Model with 25% of features accuracy is:  0.0727317826039


### 50% das features

In [182]:
clf = SGDClassifier()

features_train, labels_train, features_test, labels_test = select_features(10)

clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
acc_lm = clf.score(features_test, labels_test)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

print "Linear Model with 25% of features accuracy is: ", acc_lm

Mean squared error: 163.05
Variance score: -0.38
Linear Model with 25% of features accuracy is:  0.0705820747437


### 5% das features

In [183]:
clf = SGDClassifier()

features_train, labels_train, features_test, labels_test = select_features(5)

clf.fit(features_train, labels_train)
prediction = clf.predict(features_test)
acc_lm = clf.score(features_test, labels_test)

# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(labels_test, prediction))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(labels_test, prediction))

print "Linear Model with 25% of features accuracy is: ", acc_lm

Mean squared error: 208.60
Variance score: -0.76
Linear Model with 25% of features accuracy is:  0.0527505236468


## Cost function vs Learning rate